### This file is for addding a column of time in the energy file

In [2]:
import pandas as pd
import regex as re
import sys, os
import time
import ast
import math
from statistics import mean
from ast import literal_eval
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
import matplotlib.pyplot as plt
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from statistics import mean, median
from sklearn.tree import DecisionTreeClassifier 
from collections import Counter
# neural network
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM



2021-08-30 20:41:29.555762: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-30 20:41:29.555787: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


### Mirei attack dataset

In [5]:
from pandas import DataFrame
from pandas import concat




def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
#     print(df)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg



def get_pre_rec(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
#     print(y_pred)
#     print(y_test)
    pre=precision_score(y_test, y_pred, average='weighted')
    rec=recall_score(y_test, y_pred, average='weighted')
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred, pos_label=1)
    #print (fpr, tpr, thresholds)
    auc=metrics.auc(fpr, tpr)
    return pre, rec, auc



## Make energy to timestamp dataset

In [17]:

attacks = ["UFOnet","RouterSploit", "Mirai",  "Normal"]



for attack in attacks:
    iot_devices = ["wr940n", "archer", "camera", "cctv", "indoor", "phillips"]
    print("Actual attack is", attack)
    dataset_folder = "/home/amine/Documents/CRIM_Project/Energy-network/"+attack+"/"
    
    if attack == "UFOnet":
        iot_devices = ["archer", "cctv", "indoor"]
    
    dfObj = pd.DataFrame()

    

    for i, target in enumerate(iot_devices):
        output = "/home/amine/Documents/CRIM_Project/timeseries_energy_network/"+attack+"/"+target+"-"+attack+".csv"
        print(target)
        dataset_file =  dataset_folder+target+"-"+attack+".csv"

        if os.path.isfile(dataset_file):
            print(dataset_file)
            dataset = pd.read_csv(dataset_file, delimiter=',')
            
            
            for index, line in dataset.iterrows():
                
                val = int(len(ast.literal_eval(line['protocol'])))
                dataset.loc[index,'packet'] = val
                unique_sour = len(set(ast.literal_eval(line['source'])))
                unique_dest = len(set(ast.literal_eval(line['destination'])))
                list_length = list(ast.literal_eval(line['length']))
                if(list_length):
                    list_length.sort()
                    max_len = int(max (list_length))
                    min_len = int(min (list_length))
                    mean_len = mean (list_length)
                    mid = len(list_length) // 2
                    median_len = (list_length[mid] + list_length[~mid]) / 2
                dataset.loc[index,'unique_sour'] = unique_sour
                dataset.loc[index,'unique_dest'] = unique_dest
                dataset.loc[index,'median_len'] = median_len
                dataset.loc[index,'mean_len'] = mean_len
                dataset.loc[index,'max_len'] = max_len
                dataset.loc[index,'min_len'] = min_len
                
            dataset = dataset.drop(columns=['source', 'destination', 'info', 'length', 'timestamp', 'time'])
#             print(dataset)
            cols = dataset.columns

            dataset_series = series_to_supervised(dataset, 2)

            dataset_series ["device"] = i
            
            
#         dfObj = dfObj.append(dataset_series) 
        
            for count, c in enumerate(cols):
                dataset_series.columns = dataset_series.columns.str.replace("var"+str(count+1), c)
#             print(dataset_series.columns)
            print("writing")
            dataset_series.to_csv(output, encoding='utf-8', index=False)

            
            

        
  

Actual attack is UFOnet
archer
/home/amine/Documents/CRIM_Project/Energy-network/UFOnet/archer-UFOnet.csv
writing
cctv
/home/amine/Documents/CRIM_Project/Energy-network/UFOnet/cctv-UFOnet.csv
writing
indoor
/home/amine/Documents/CRIM_Project/Energy-network/UFOnet/indoor-UFOnet.csv
writing
Actual attack is RouterSploit
wr940n
/home/amine/Documents/CRIM_Project/Energy-network/RouterSploit/wr940n-RouterSploit.csv
writing
archer
/home/amine/Documents/CRIM_Project/Energy-network/RouterSploit/archer-RouterSploit.csv
writing
camera
/home/amine/Documents/CRIM_Project/Energy-network/RouterSploit/camera-RouterSploit.csv
writing
cctv
/home/amine/Documents/CRIM_Project/Energy-network/RouterSploit/cctv-RouterSploit.csv
writing
indoor
/home/amine/Documents/CRIM_Project/Energy-network/RouterSploit/indoor-RouterSploit.csv
writing
phillips
/home/amine/Documents/CRIM_Project/Energy-network/RouterSploit/phillips-RouterSploit.csv
writing
Actual attack is Mirai
wr940n
/home/amine/Documents/CRIM_Project/Ene

## Mixte each attack with the normal

In [18]:
attacks = ["Mirai", "RouterSploit", "UFOnet", "Normal"]



for attack in attacks:
    
    iot_devices = ["wr940n", "archer", "camera", "cctv", "indoor", "phillips"]
    print("Actual attack is", attack)
    dataset_folder = "/home/amine/Documents/CRIM_Project/Energy-network/"+attack+"/"
    
    if attack == "UFOnet":
        iot_devices = ["archer", "cctv", "indoor"]
    
    dfObj = pd.DataFrame()

    for i, target in enumerate(iot_devices):
        dataset_file = "/home/amine/Documents/CRIM_Project/timeseries_energy_network/"+attack+"/"+target+"-"+attack+".csv"
        
        
        dataset = pd.read_csv(dataset_file, delimiter=',')
        
        
        dfObj = dfObj.append(dataset) 
        
#     print(dfObj)
    
    
    dfObj.to_csv("/home/amine/Documents/CRIM_Project/timeseries_energy_network/"+attack+"/"+attack+"-all.csv", encoding='utf-8', index=False)
    


Actual attack is Mirai
Actual attack is RouterSploit
Actual attack is UFOnet
Actual attack is Normal


In [19]:
attacks = ["Mirai", "RouterSploit", "UFOnet"]



for attack in attacks:    
    dataset_folder = "/home/amine/Documents/CRIM_Project/timeseries_energy_network/"+attack+"/"
    mirei_input_folder = dataset_folder+attack+"-all.csv"
    
    mirei_output_folder = dataset_folder+"pre_ml-"+attack+".csv"

    dataset_attack = pd.read_csv(mirei_input_folder, delimiter=',')
    dataset_normal = pd.read_csv("/home/amine/Documents/CRIM_Project/timeseries_energy_network/Normal/Normal-all.csv", delimiter=',')

    dataset_attack['target'] = 1
    dataset_normal['target'] = 0
    merged_data = pd.concat([dataset_attack,dataset_normal], axis=0)
    
    
    merged_data.to_csv(mirei_output_folder, index=False)

### Dummies for protocol variables

In [21]:
attacks = ["Mirai", "RouterSploit", "UFOnet"]
from collections import Counter


for attack in attacks:  
    dataset_folder = "/home/amine/Documents/CRIM_Project/timeseries_energy_network/"+attack+"/"
    mirei_input_folder = dataset_folder+"pre_ml-"+attack+".csv"
    mirei_output_folder = dataset_folder+"pre_ml_dummies-"+attack+".csv"
    dataset = pd.read_csv(mirei_input_folder, delimiter=',')
    
    
    columns_dum = ['protocol(t-2)', 'protocol(t-1)', 'protocol(t)']
    
    for col in columns_dum:
        bridge_types = []
        for index, line in dataset.iterrows():
            
            val = list(set(ast.literal_eval(line[col])))
            bridge_types.extend(val)
            bridge_types = list(set(bridge_types))
        
                
        for col_bridge in bridge_types:
            dataset[col+'_'+col_bridge] = 0 
            
        for index, line in dataset.iterrows():
            val = list(set(ast.literal_eval(line[col])))
            
            
            occurrences = dict(Counter(val))
            
            for pro, occ in occurrences.items():
                
                dataset.loc[index, col+'_'+pro] = occ
        
        del dataset[col]

            
            
#         print(dataset)
    dataset.to_csv(mirei_output_folder, index=False)
        

## Apply ML algrithms (random forest, logictic regression, decision tree)

In [6]:
attacks = ["Mirai", "RouterSploit", "UFOnet"]



for attack in attacks:
    dataset_folder = "/home/amine/Documents/CRIM_Project/timeseries_energy_network/"+attack+"/"
    input_folder = dataset_folder+"pre_ml_dummies-"+attack+".csv"
    dataset = pd.read_csv(input_folder, delimiter=',')
    
    target='target'

#     features = list(dataset.columns)
#     print(list(dataset.columns))
    dataset = dataset[~dataset[target].isnull()]
    dataset=dataset.reset_index(drop=True)
    
    
    y = dataset[target].astype(int)
#     print(y.value_counts())
    features = list(dataset.columns)
    features.remove(target)
#     energy_features = ["energy(t)", "energy(t-1)", "energy(t-2)"]
#     for e in energy_features:
#         features.remove(e)
    x = dataset[features]
    
    folds = StratifiedKFold(n_splits=10, shuffle=True)
    pre_rf = []
    pre1_rf = []
    pre2_rf = []
    rec_rf = []
    rec1_rf = []
    rec2_rf = []
    auc_rf = []
    auc1_rf = []
    auc2_rf = []
    for train_index, test_index in folds.split(x,y):
        x_train=x.iloc[train_index]
        x_test=x.iloc[test_index]
        y_train=y.iloc[train_index]
        y_test =  y.iloc[test_index]
        sm = SMOTE(random_state=42, sampling_strategy='auto')
        x_train, y_train = sm.fit_resample(x_train, y_train)



        pre, rec, auc = get_pre_rec(RandomForestClassifier(), x_train, x_test, y_train, y_test)
        pre1, rec1, auc1 = get_pre_rec(LogisticRegression(), x_train, x_test, y_train, y_test)
        pre2, rec2, auc2 = get_pre_rec(DecisionTreeClassifier(), x_train, x_test, y_train, y_test)
        pre_rf.append (pre)
        rec_rf.append (rec)
        auc_rf.append (auc)
        pre1_rf.append (pre1)
        rec1_rf.append (rec1)
        auc1_rf.append (auc1)
        pre2_rf.append (pre2)
        rec2_rf.append (rec2)
        auc2_rf.append (auc2)


    print ("random forest {:.2f}".format(median(pre_rf)*100), "{:.2f}".format(median(rec_rf)*100), "{:.2f}".format(median(auc_rf)*100) )
    print ("logistic regression{:.2f}".format(median(pre1_rf)*100), "{:.2f}".format(median(rec1_rf)*100), "{:.2f}".format(median(auc1_rf)*100))
    print ("decision tree {:.2f}".format(median(pre2_rf)*100), "{:.2f}".format(median(rec2_rf)*100), "{:.2f}".format(median(auc2_rf)*100))

/home/amine/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/amine/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
 

random forest 98.16 98.16 97.51
logistic regression78.30 64.96 71.13
decision tree 98.15 98.15 97.49


/home/amine/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/amine/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
 

random forest 97.97 97.95 98.00
logistic regression55.98 56.25 55.57
decision tree 97.48 97.47 97.50


/home/amine/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/amine/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
 

random forest 99.93 99.93 99.91
logistic regression84.61 64.88 65.69
decision tree 99.91 99.91 99.85


## Neural Network

In [ ]:
attacks = ["Mirai", "RouterSploit", "UFOnet"]



for attack in attacks:
    print(attack)
    dataset_folder = "/home/amine/Documents/CRIM_Project/energy/"+attack+"/"
    input_folder = dataset_folder+"pre_ml-"+attack+".csv"
    dataframe = pd.read_csv(input_folder, delimiter=',')
    # load dataset
    dataset = dataframe.values
    # split into input (X) and output (Y) variables
    X = dataset[:,0:6].astype(float)
    Y = dataset[:,6]


    # encode class values as integers
    encoder = LabelEncoder()
    encoder.fit(Y)
    encoded_Y = encoder.transform(Y)


    # baseline model
    def create_baseline():
        # create model
        model = Sequential()
        model.add(Dense(6, input_dim=6, activation='relu'))
        model.add(Dense(1, activation='sigmoid'))
        # Compile model
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['Recall', "Precision"])
        return model



    # evaluate model with standardized dataset
    estimator = KerasClassifier(build_fn=create_baseline, epochs=5, batch_size=1, verbose=2)
    kfold = StratifiedKFold(n_splits=10, shuffle=True)
    results = cross_val_score(estimator, X, encoded_Y, cv=kfold)
    print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    


Mirai
Epoch 1/5
41525/41525 - 20s - loss: 0.5573 - recall: 0.9956 - precision: 0.7464
Epoch 2/5
41525/41525 - 19s - loss: 0.5133 - recall: 1.0000 - precision: 0.7465
Epoch 3/5
41525/41525 - 20s - loss: 0.4573 - recall: 0.9982 - precision: 0.8017
Epoch 4/5
41525/41525 - 20s - loss: 0.4164 - recall: 0.9956 - precision: 0.8254
Epoch 5/5
41525/41525 - 20s - loss: 0.4019 - recall: 0.9918 - precision: 0.8265
4614/4614 - 2s - loss: 0.3936 - recall: 0.9991 - precision: 0.8290
Epoch 1/5


/home/amine/.local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/amine/.local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/amine/.local/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 90, in __call__
    score = scorer(estimator, *args, **kwargs)
  File "/home/amine/.local/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 397, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/home/amine/.local/lib/python3.9/site-packages/keras/wrappers/scikit_learn.py", line 305, in score
    raise ValueError('The model is not configured to compute accuracy. '
ValueError: The model is not configured to compute accuracy. You should pass `metrics=["accuracy"]`

41525/41525 - 21s - loss: 0.5562 - recall: 0.9962 - precision: 0.7477
Epoch 2/5
41525/41525 - 22s - loss: 0.5255 - recall: 0.9930 - precision: 0.7575
Epoch 3/5
41525/41525 - 21s - loss: 0.5178 - recall: 0.9835 - precision: 0.7567
Epoch 4/5
41525/41525 - 21s - loss: 0.5156 - recall: 0.9790 - precision: 0.7563
Epoch 5/5
41525/41525 - 21s - loss: 0.5139 - recall: 0.9784 - precision: 0.7559
4614/4614 - 2s - loss: 0.5174 - recall: 0.9988 - precision: 0.7568
Epoch 1/5


/home/amine/.local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/amine/.local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/amine/.local/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 90, in __call__
    score = scorer(estimator, *args, **kwargs)
  File "/home/amine/.local/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 397, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/home/amine/.local/lib/python3.9/site-packages/keras/wrappers/scikit_learn.py", line 305, in score
    raise ValueError('The model is not configured to compute accuracy. '
ValueError: The model is not configured to compute accuracy. You should pass `metrics=["accuracy"]`

41525/41525 - 21s - loss: 0.5569 - recall: 1.0000 - precision: 0.7465
Epoch 2/5
41525/41525 - 20s - loss: 0.5158 - recall: 1.0000 - precision: 0.7465
Epoch 3/5
41525/41525 - 20s - loss: 0.5079 - recall: 1.0000 - precision: 0.7465
Epoch 4/5
41525/41525 - 20s - loss: 0.5070 - recall: 1.0000 - precision: 0.7469
Epoch 5/5
41525/41525 - 20s - loss: 0.5062 - recall: 1.0000 - precision: 0.7472
4614/4614 - 2s - loss: 0.4992 - recall: 1.0000 - precision: 0.7497
Epoch 1/5


...

234096/234096 - 177s - loss: 0.0493 - recall: 0.9955 - precision: 0.9805
Epoch 2/5
234096/234096 - 168s - loss: 0.0273 - recall: 0.9891 - precision: 0.9954
Epoch 3/5
234096/234096 - 150s - loss: 0.0270 - recall: 0.9880 - precision: 0.9965
Epoch 4/5
234096/234096 - 156s - loss: 0.0272 - recall: 0.9879 - precision: 0.9968
Epoch 5/5
234096/234096 - 162s - loss: 0.0273 - recall: 0.9876 - precision: 0.9973
26011/26011 - 15s - loss: 0.0261 - recall: 0.9996 - precision: 0.9847
Epoch 1/5


/home/amine/.local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/amine/.local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/amine/.local/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 90, in __call__
    score = scorer(estimator, *args, **kwargs)
  File "/home/amine/.local/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 397, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/home/amine/.local/lib/python3.9/site-packages/keras/wrappers/scikit_learn.py", line 305, in score
    raise ValueError('The model is not configured to compute accuracy. '
ValueError: The model is not configured to compute accuracy. You should pass `metrics=["accuracy"]`

234096/234096 - 174s - loss: 0.0425 - recall: 0.9982 - precision: 0.9843
Epoch 2/5
234096/234096 - 126s - loss: 0.0219 - recall: 0.9985 - precision: 0.9929
Epoch 3/5
234096/234096 - 118s - loss: 0.0217 - recall: 0.9985 - precision: 0.9929
Epoch 4/5
234096/234096 - 118s - loss: 0.0217 - recall: 0.9985 - precision: 0.9930
Epoch 5/5
234096/234096 - 117s - loss: 0.0217 - recall: 0.9986 - precision: 0.9930
26011/26011 - 11s - loss: 0.0176 - recall: 0.9995 - precision: 0.9930
Epoch 1/5


/home/amine/.local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/amine/.local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/amine/.local/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 90, in __call__
    score = scorer(estimator, *args, **kwargs)
  File "/home/amine/.local/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 397, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/home/amine/.local/lib/python3.9/site-packages/keras/wrappers/scikit_learn.py", line 305, in score
    raise ValueError('The model is not configured to compute accuracy. '
ValueError: The model is not configured to compute accuracy. You should pass `metrics=["accuracy"]`

234096/234096 - 118s - loss: 0.0910 - recall: 0.9889 - precision: 0.9747
Epoch 2/5
234096/234096 - 115s - loss: 0.0860 - recall: 0.9881 - precision: 0.9765
Epoch 3/5
234096/234096 - 107s - loss: 0.0857 - recall: 0.9881 - precision: 0.9764
Epoch 4/5
